In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
def retrieve_content(url):
    try:
        response = requests.get(url, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        questions = soup.find_all('div', class_='bix-div-container')
        data = []
        
        for question in questions:
            q_number = question.find('div', class_='bix-td-qno').text.strip()
            q_text = question.find('div', class_='bix-td-qtxt').text.strip()
            options_div = question.find('div', class_='bix-tbl-options')
            
            options = []
            if options_div:
                for option in options_div.find_all('div', class_='bix-opt-row'):
                    option_text = option.text.strip()
                    if option_text:
                        options.append(option_text)
            
            answer = question.find('input', class_='jq-hdnakq')['value']
            explanation = question.find('div', class_='bix-div-answer').text.strip() if question.find('div', class_='bix-div-answer') else 'No explanation provided'
            
            data.append({
                'Question Number': q_number,
                'Question Text': q_text,
                'Options': '; '.join(options),
                'Answer': answer,
                'Explanation': explanation
            })
        
        return data
    except requests.RequestException as e:
        print(f"Error fetching content from {url}: {e}")
        return []

In [3]:
def create_or_append_xls_file(data, filename):
    try:
        if os.path.exists(filename):
            df_existing = pd.read_excel(filename)
            df_new = pd.DataFrame(data)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            df_combined.to_excel(filename, index=False)
        else:
            df_new = pd.DataFrame(data)
            df_new.to_excel(filename, index=False)
        
        #print(f"Excel file '{filename}' updated successfully.")
    except Exception as e:
        print(f"Error creating or appending Excel file '{filename}': {e}")

In [4]:
def handle_pages(base_url, total_pages, folder_name):
    # Create the new directory if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        #print(f"Created directory: {folder_name}")
    
    for page_number in range(1, total_pages + 1):
        # Adjust the page URL for the first page or subsequent pages
        if page_number == 1:
            page_url = base_url
        else:
            page_url = f"{base_url}/{str(page_number).zfill(6)}"
        
        #print(f"Processing page: {page_url}")
        
        # Retrieve content from the page
        data = retrieve_content(page_url)
        
        if data:
            # Construct the file name using the folder path and base URL
            file_name = os.path.join(folder_name, base_url.split('/')[-2] + '.xlsx')
            
            # Create the file or append to it
            create_or_append_xls_file(data, file_name)
        else:
            print(f"No content retrieved for {page_url}")

    print(f"All files saved in the directory: {folder_name}")

In [5]:
import requests
from bs4 import BeautifulSoup

def find_folders(folder_ls, url):
    try:
        # Fetch the content from the URL
        response = requests.get(url, verify=False)  # Set verify=False to ignore SSL warnings
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all folder links
        folders = []
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if f'/{folder_ls}/' in href and href not in folders:
                folders.append(href)
        
        return folders

    except requests.RequestException as e:
        print(f"Error fetching content from {url}: {e}")
        return []

In [6]:
def process_all_folders_and_subfolders(url, folder_name, total_pages=7):
    # Find and process all folders
    folders = find_folders(folder_name, url)

    for folder in folders:
        print(f"Processing folder: {folder}")
        
        # Find subfolders within the current folder
        subfolders = find_folders(folder_name, folder)
        
        # If no subfolders are found, handle just the main folder
        if not subfolders:
            handle_pages(folder, total_pages, folder_name)
        else:
            # Process each subfolder individually
            for subfolder in subfolders:
                print(f"Processing subfolder: {subfolder}")
                handle_pages(subfolder, total_pages, folder_name)

        # After processing all subfolders of the current folder, move to the next folder
        print(f"Finished processing folder: {folder}")
        break

    print("All folders and subfolders processed successfully.")


In [7]:
def check_for_duplicates(file_path):
    """Check for duplicate entries in the Excel file based on 'Question Text'."""
    try:
        # Read the data from the Excel file
        df = pd.read_excel(file_path)

        # Check for duplicate questions
        if 'Question Text' not in df.columns:
            print("The expected column 'Question Text' is missing from the file.")
            return

        # Identify duplicates
        duplicates = df[df.duplicated(subset=['Question Text'], keep=False)]

        if duplicates.empty:
            print("No duplicate questions found.")
        else:
            print(f"Found {len(duplicates)} duplicate entries.")
            print(duplicates[['Question Number', 'Question Text']])
            # Optionally, save duplicates to a separate file
            duplicates.to_excel('duplicates_found.xlsx', index=False)
            print("Duplicate entries have been saved to 'duplicates_found.xlsx'.")

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [8]:
def remove_duplicates_from_file(file_path):
    """Read an .xlsx file, remove duplicate rows, and save it back."""
    try:
        df = pd.read_excel(file_path)
        
        # Remove duplicate rows based on 'Question Number' and 'Question Text'
        df_cleaned = df.drop_duplicates(subset=['Question Number', 'Question Text'])
        
        # Save the cleaned DataFrame back to the file
        df_cleaned.to_excel(file_path, index=False)
        print(f"Duplicates removed and file saved: {file_path}")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

In [9]:
def process_all_xlsx_files(folder_path):
    """Process all .xlsx files in the specified folder."""
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file_name)
            remove_duplicates_from_file(file_path)

In [10]:
def search_xlsx_files(directory):
    """Search for all .xlsx files in the given directory, excluding temporary files."""
    xlsx_files = [os.path.join(directory, f) for f in os.listdir(directory)
                  if f.endswith('.xlsx') and not f.startswith('~$')]
    return xlsx_files

In [11]:
def read_data_from_file(file_path):
    """Read data from an .xlsx file and return it as a DataFrame."""
    try:
        df = pd.read_excel(file_path)
        return df
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return pd.DataFrame()  # Return an empty DataFrame
    except PermissionError:
        print(f"Permission denied: {file_path}")
        return pd.DataFrame()  # Return an empty DataFrame

In [12]:
def combine_data(xlsx_files, output_file):
    all_data = []
    unique_questions = set()  # To keep track of unique questions

    for file_path in xlsx_files:
        folder_name = os.path.splitext(os.path.basename(file_path))[0]
        df = read_data_from_file(file_path)
        
        for index, row in df.iterrows():
            question = row.get('Question Text', '')
            if question and question not in unique_questions:
                unique_questions.add(question)
                all_data.append({
                    'Question Number': row.get('Question Number', ''),
                    'Question Text': question,
                    'Options': row.get('Options', ''),
                    'Answer': row.get('Answer', ''),
                    'Explanation': row.get('Explanation', ''),
                    'Folder Name': folder_name
                })
    
    # Create a DataFrame from the consolidated data
    consolidated_df = pd.DataFrame(all_data)

    # Write the DataFrame to an .xlsx file using openpyxl
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        consolidated_df.to_excel(writer, index=False, sheet_name='Consolidated Data')

In [13]:
def count_questions_per_topic(file_path):
    """Count the number of questions per topic and the total number of questions in the Excel file."""
    try:
        # Read the data from the Excel file
        df = pd.read_excel(file_path)

        # Check if the required columns exist
        if 'Folder Name' not in df.columns:
            print("The expected column 'Folder Name' is missing from the file.")
            return

        # Group by 'Folder Name' and count the number of questions
        topic_counts = df.groupby('Folder Name').size().reset_index(name='Question Count')

        # Print the number of questions per topic
        print("Number of questions per topic:")
        print(topic_counts)

        # Calculate total number of questions
        total_questions = df.shape[0]

        # Print the total number of questions
        print(f"\nTotal number of questions: {total_questions}")

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [15]:
folder_list = ["aptitude", "data-interpretation", "verbal-ability", "logical-reasoning", "verbal-reasoning", "non-verbal-reasoning"]

for folder in folder_list:
    url = f'https://www.indiabix.com/{folder}/questions-and-answers/'
    folder_name = folder  # Name of the directory to save all files
    folder_path = "./{folder_name}"

    # Call the function to process folders and subfolders
    process_all_folders_and_subfolders(url, folder_name)

c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing folder: https://www.indiabix.com/data-interpretation/questions-and-answers/


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing subfolder: https://www.indiabix.com/data-interpretation/questions-and-answers/
Processing page: https://www.indiabix.com/data-interpretation/questions-and-answers/


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Excel file 'non-verbal-reasoning\cubes-and-dice.xlsx' updated successfully.
Processing page: https://www.indiabix.com/non-verbal-reasoning/cubes-and-dice//000006


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


No content retrieved for https://www.indiabix.com/non-verbal-reasoning/cubes-and-dice//000006
Processing page: https://www.indiabix.com/non-verbal-reasoning/cubes-and-dice//000007
No content retrieved for https://www.indiabix.com/non-verbal-reasoning/cubes-and-dice//000007
All files saved in the directory: non-verbal-reasoning
Finished processing folder: https://www.indiabix.com/non-verbal-reasoning/questions-and-answers/
All folders and subfolders processed successfully.


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [21]:
folder_list = ["aptitude", "data-interpretation", "verbal-ability", "logical-reasoning", "verbal-reasoning", "non-verbal-reasoning"]

for folder in folder_list:
    
    # Directory containing .xlsx files
    directory = f'./{folder}/'  # Correctly format the directory path

    # Search for .xlsx files
    xlsx_files = search_xlsx_files(directory)

    # Output .xlsx file
    output_file = f'{folder}.xlsx'  # Correctly format the output file name

    # Consolidate data and write to the output file
    combine_data(xlsx_files, output_file)

    # Path to the consolidated Excel file
    file_path = output_file

    # Count questions per topic in the file
    count_questions_per_topic(file_path)


Number of questions per topic:
                    Folder Name  Question Count
0         alligation-or-mixture              15
1                          area              15
2                       average              15
3              bankers-discount              13
4             boats-and-streams              15
5                      calendar              15
6                    chain-rule              15
7                         clock              19
8             compound-interest              15
9              decimal-fraction              30
10          height-and-distance               6
11                    logarithm              15
12                      numbers              35
13       odd-man-out-and-series              14
14                  partnership              14
15                   percentage              15
16  permutation-and-combination              14
17            pipes-and-cistern              15
18                  probability              15
19       

In [22]:
folder_list = ["c-programming", "cpp-programming", "c-sharp-programming", "java-programming"]

for folder in folder_list:
    url = f'https://www.indiabix.com/{folder}/questions-and-answers/'
    folder_name = folder  # Name of the directory to save all files
    folder_path = "./{folder_name}"

    # Call the function to process folders and subfolders
    process_all_folders_and_subfolders(url, folder_name)

    directory = f'./{folder}/'  # Correctly format the directory path

    # Search for .xlsx files
    xlsx_files = search_xlsx_files(directory)

    # Output .xlsx file
    output_file = f'{folder}.xlsx'  # Correctly format the output file name

    # Consolidate data and write to the output file
    combine_data(xlsx_files, output_file)

    # Path to the consolidated Excel file
    file_path = output_file

    # Count questions per topic in the file
    count_questions_per_topic(file_path)


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing folder: https://www.indiabix.com/c-programming/questions-and-answers/


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing subfolder: https://www.indiabix.com/c-programming/questions-and-answers/
Processing page: https://www.indiabix.com/c-programming/questions-and-answers/


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

No content retrieved for https://www.indiabix.com/java-programming/java-lang-class//000005
Processing page: https://www.indiabix.com/java-programming/java-lang-class//000006


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


No content retrieved for https://www.indiabix.com/java-programming/java-lang-class//000006
Processing page: https://www.indiabix.com/java-programming/java-lang-class//000007


c:\Users\LENOVO\anaconda3\envs\Terna\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.indiabix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


No content retrieved for https://www.indiabix.com/java-programming/java-lang-class//000007
All files saved in the directory: java-programming
Finished processing folder: https://www.indiabix.com/java-programming/questions-and-answers/
All folders and subfolders processed successfully.
Number of questions per topic:
                        Folder Name  Question Count
0                        assertions               5
1   declarations-and-access-control              18
2                        exceptions              10
3                      flow-control               4
4               garbage-collections               8
5                     inner-classes               7
6                   java-lang-class               4
7             language-fundamentals              14
8           objects-and-collections              12
9         operators-and-assignments              15
10                          threads              15

Total number of questions: 112


In [108]:
def find_last_non_empty_row(file_path):
    """Find the last non-empty row in an .xlsx file."""
    try:
        df = pd.read_excel(file_path)
        if not df.empty:
            # Find the last non-empty row index
            last_row_index = df.last_valid_index()
            return last_row_index
        else:
            return None
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

In [122]:

def process_all_xlsx_files_in_folder(folder_path):
    """Process all .xlsx files in the given folder and find the last non-empty row."""
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file_name)
            last_row_index = find_last_non_empty_row(file_path)
            if last_row_index is not None:
                print(f"{file_name} - No of Questions : {last_row_index-1}")
            else:
                print(f"File: {file_name} is empty or could not be processed.")

In [123]:
# Path to the current folder containing .xlsx files
current_folder = '.'  # Replace 'your_folder_name' with your folder path

# Process all .xlsx files in the current folder
process_all_xlsx_files_in_folder(current_folder)

aptitude.xlsx - No of Questions : 591
c-programming.xlsx - No of Questions : 114
c-sharp-programming.xlsx - No of Questions : 336
cpp-programming.xlsx - No of Questions : 103
data-interpretation.xlsx - No of Questions : 22
java-programming.xlsx - No of Questions : 110
logical-reasoning.xlsx - No of Questions : 355
verbal-ability.xlsx - No of Questions : 407
verbal-reasoning.xlsx - No of Questions : 369
